In [1]:
import time
import logging

import krpc
import numpy as np
from IPython.display import clear_output

from UPG import UPG, Status
import utils
import Apollo


In [2]:
conn = krpc.connect()
space_center = conn.space_center
vessel = space_center.active_vessel
ap = vessel.auto_pilot
body = vessel.orbit.body
body_frame = body.reference_frame
flight = vessel.flight(body_frame)

# body constants
r_e = body.equatorial_radius
g_0 = body.surface_gravity

In [3]:
lon = -60.0
lat = 0.0
final_velocity = 3.    # The target velocity for UPG. Note that is this not the touch down velocity.
final_height = 50. # The target height above target when upg ends.
final_acc = 1
min_throttle = 0.1  # Minimum throttle. This must be exactly equal to your engine's min throttle.
t_1 = 10.    # First burn time, in secondes.
rated_burn_time = 960   # Engine's rated burn time, or the time you want to burn at most.


# if you find your craft turning too slow, alter the following values.
ap.stopping_time = (.9, .9, .9)
# ap.deceleration_time = (3., 3., 3.)
ap.attenuation_angle = (0.5, 0.5, 0.5)

In [4]:
# target vector, with final height added.
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height + final_height, body_frame))

# target reference frame.
target_frame = utils.target_reference_frame(space_center, body, lat, lon)
# get final velocity and up vector in body frame
up = r_target / np.linalg.norm(r_target)
v_target = -final_velocity * up

# draw the target
line = conn.drawing.add_line((0., 0., 0.), (20., 0., 0.), target_frame)
line.thickness = 2

In [5]:
vacuum_specific_impulse = conn.add_stream(getattr, vessel,
                                          'vacuum_specific_impulse')
max_thrust = conn.add_stream(getattr, vessel, 'available_thrust')
current_thrust = conn.add_stream(getattr, vessel, 'thrust')
mass = conn.add_stream(getattr, vessel, 'mass')
situation = conn.add_stream(getattr, vessel, 'situation')
ut = conn.add_stream(getattr, space_center, 'ut')
height = conn.add_stream(getattr, flight, 'surface_altitude')
mean_altitude = conn.add_stream(getattr, flight, 'mean_altitude')
speed = conn.add_stream(getattr, flight, 'speed')
vertical_speed = conn.add_stream(getattr, flight, 'vertical_speed')
velocity = conn.add_stream(vessel.velocity, body_frame)
position = conn.add_stream(vessel.position, body_frame)

In [6]:
# get initial values
v_0 = utils.vector(velocity())
r_0 = utils.vector(position())
# initial guess
pv = -v_0 / np.linalg.norm(v_0)
pr = np.zeros((3, 1))
t_f = np.array([[rated_burn_time]]) / 2

print("Initiating UPG")
upg = UPG(r_0=r_e, g_0=g_0, r_target=r_target, v_target=v_target, r_t=r_0, v_t=v_0,
    mass=mass(), max_thrust=max_thrust(), min_throttle=min_throttle, 
    specific_impulse=vacuum_specific_impulse(), k=0., t_0=ut(), t_1=t_1, p_r_0=pr, p_v_0=pv,
    t_f=t_f, t_2_bound=[5, rated_burn_time / 20.])

print("Evaluating t2...")
upg.solve_t_2()


Initiating UPG
Evaluating t2...
Terminate program if it stucks at solving t_2.


In [7]:
# PDI determination
print("Waiting for powered descent initial")
ap.reference_frame = body_frame
ap.engage()
guidance_interval = 0.1




while True:
    v_0 = utils.vector(velocity())
    r_0 = utils.vector(position())
    
    if upg.status == Status.PDI:
        upg.update_state(r_t=r_0, v_t=v_0, mass=mass(),
               max_thrust=max_thrust(), t_0=ut())
        upg.update_start_time(ut())
        upg.solve(mode='soft')
        r_guidance = utils.downrange(r_0, upg.r_final, body, body_frame)
        r_togo = utils.downrange(r_0, r_target, body, body_frame)
        print("guidance downrange: {0:,.2f}m, target downrange: {1:,.2f}m"
            .format(r_guidance, r_togo))
        ap.target_direction = upg.thrust_direction
        # start descent when overshoot the target with margin of 1km.
        if r_guidance > r_togo + 1000.:
            print("descent start!")
            upg.status = Status.Powered_descent
            t_f = 1.2 * upg.t_f
            t_f = t_f + ut()    # convert to ut
            apdg = Apollo.APDG(r_target, final_velocity, final_acc, t_f, ut(), up, mass(), r_0, v_0, g_0, 8.)
            continue
        

    elif upg.status == Status.Powered_descent:
        apdg.update(r_0, v_0, mass(), ut())
        apdg.compute()

        vessel.control.throttle = utils.thrust2throttle(
        apdg.thrust, max_thrust(), min_throttle)
        ap.target_direction = apdg.thrust_direction

        print("t_go: {:.2f} s".format(apdg.t_go), end=' ')
        r_go = np.linalg.norm(r_target - r_0)
        print("distance to land: {:,.2f} m".format(r_go))
        print("thrust error: {:.2f} kN".format((vessel.thrust - apdg.thrust) / 1e3))
        print("AP direction error {:.2f}".format(ap.error))

        clear_output(wait=True)
        if apdg.t_go < 0.5 or r_go < 10 or height() < 100:
            print("APDG disengaged")
            break
    clear_output(wait=True)
    time.sleep(guidance_interval)

while True:
    ap.target_direction = tuple(-i for i in velocity())
    
    vs = vertical_speed()
    spd = speed()
    sin_g = vs/spd
    h = mean_altitude()
    a_t = - (1. / sin_g) * (((final_velocity ** 2 - vs ** 2)/ (2 * (target_height - h))) + g_0)
    vessel.control.throttle = utils.thrust2throttle(
        mass() * a_t, max_thrust(), min_throttle)
    if situation() == space_center.VesselSituation.landed \
            or situation() == space_center.VesselSituation.splashed\
            or vertical_speed() > -0.5:
        print("landed!")
        break
ap.disengage()
vessel.control.throttle = 0.

r_0 = utils.vector(position())
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height, body_frame))
print("Landing precision: {:.2f}m".format(utils.downrange(r_0, r_target, body, body_frame)))


APDG disengaged
landed!
Landing precision: 5.93m
